Want to put standard kernels here

This example shows you how to use mlflow tracking APIs together with Azure Machine Learning services for storing your metrics and artifacts, from local Notebook run. You'll learn how to:

1. Set up MLflow tracking URI so as to use Azure ML
2. Create experiment
3. Train a model on ML compute while logging metrics and artifacts
4. View your experiment within your Azure ML Workspace in Azure Portal.

In [ ]:
# need to doctor your own train.py file from lucky's that preserves the necessary info

In [ ]:
# set tracking URI
# create experiment
# create train and test sets
# train while logging metrics and artifacts. train script should use mlflow to log model HP values, 
#   errors, save final D_init, W, embedding after each epoch

In [28]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace = ws, name = cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [29]:
import sys, os
import mlflow
import mlflow.azureml

import azureml.core
from azureml.core import Workspace

ws = Workspace.from_config()

print("SDK version:", azureml.core.VERSION)
print("MLflow version:", mlflow.version.VERSION)
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

SDK version: 1.27.0
MLflow version: 1.15.0
ReactionModelling
ResearchProj
uksouth
4ba7b086-969d-41c4-a647-2784cde6af4b


In [32]:
# set tracking URI
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

# create mlflow experiment
experiment_name = "train-tsgen"
mlflow.set_experiment(experiment_name)

# create backend config object
backend_config = {"COMPUTE": "cpucluster", "USE_CONDA": False}

In [33]:
# submit run
remote_mlflow_run = mlflow.projects.run(uri=".", 
                                    parameters={"alpha":0.3},
                                    backend = "azureml",
                                    backend_config = backend_config,
                                    synchronous=True)

Class AzureMLProjectBackend: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


ExecutionException: Could not find main among entry points [] or interpret main as a runnable script. Supported script file extensions: ['.py', '.sh']

In [ ]:
# view metrics and artifacts in your workspace
run.get_metrics()

# once run complete
# the model folder produced from the run is registered. This includes the MLmodel file, model.pkl and the conda.yaml.
run.register_model(model_name = 'my-model', model_path = 'model')

# then view registered model in worksapce with aml studio

In [11]:
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, loguniform

In [12]:
ps = RandomParameterSampling(
    {
        '--batch_size': choice(), # 8
        '--hidden_size': choice(), # 128
        '--layers': choice(), #2
        '--iterations': choice() # 3
    }
)

# BanditPolicy checks job every (evaluation_interval) number of iterations terminating the job if primary metric outside of slack_factor
early_term_policy = BanditPolicy(evaluation_interval = 2, slack_factor = 0.1)

# HyperDriveConfig
hdc = HyperDriveConfig(estimator = est, hyperparameter_sampling = ps, 
                       policy = early_term_policy, primary_metric_name = 'Accuracy',
                       primary_metric_goal = PrimaryMetricGoal.MAXIMIZE, 
                       max_total_runs = 20, max_concurrent_runs = 4)


HyperDriveConfigException: HyperDriveConfigException:
	Message: Please specify an input for choice.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Please specify an input for choice.",
        "details_uri": "https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.hyperdriveconfig?view=azure-ml-py",
        "target": "options",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentBlankOrEmpty"
            }
        }
    }
}